In [1]:
import os
import sys
import re
import time
from pathlib import Path
import pdfplumber
import pytesseract
from pdf2image import convert_from_path
import warnings
import logging
import pandas as pd
import numpy as np
import json
from matplotlib import pyplot as plt

sys.path.append('../python')
warnings.filterwarnings('default')
logging.getLogger("pdfminer").setLevel(logging.ERROR)

DATA_DIR = Path('../../raw_data/cpc')

import api

import nltk
from nltk.corpus import stopwords

# Download stopwords if you haven't already
nltk.download('stopwords')

# Load English stopwords
stop_words = set(stopwords.words('english'))


/Users/ekung/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/ekung/Library/Python/3.9/lib/python/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
[nltk_data] Downloading package stopwords to /Users/ekung/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
meetings_meta_df = pd.read_csv("../../intermediate_data/cpc/meetings_metadata.csv")

In [3]:
for idx, row in meetings_meta_df.iterrows():
    year = row['year']
    date = row['date']
    print(f"{date}... ", end='')

    # Agenda full text
    try:
        with open(f"../../intermediate_data/cpc/{year}/{date}/agenda-manual-override.txt", 'r') as f:
            agenda_text = f.read()
    except:
        with open(f"../../intermediate_data/cpc/{year}/{date}/agenda.txt", 'r') as f:
            agenda_text = f.read()
    # Agenda item summaries
    try:
        with open(f"../../intermediate_data/cpc/{year}/{date}/agenda-item-summaries-manual-override.txt", 'r') as f:
            agenda_items = f.read()
    except:
        with open(f"../../intermediate_data/cpc/{year}/{date}/agenda-item-summaries.txt", 'r') as f:
            agenda_items = f.read()

    # Construct data frame with year, date, item_no, title, summary
    items = agenda_items.split('------')
    df = []
    for item in items:
        if len(item.strip())>0:
            parts = item.strip().split('\n')
            item_no = re.search(r'ITEM NO:\s*(.*)', parts[0].strip()).group(1)
            title = re.search(r'TITLE:\s*(.*)', parts[1].strip()).group(1)
            summary = re.search(r'SUMMARY:\s*(.*)', parts[2].strip()).group(1)
            df.append({
                'year': year,
                'date': date,
                'item_no': item_no,
                'title': title,
                'summary': summary
            })
    # Find the start line of each item
    agenda_text_lines = agenda_text.split('\n')
    for i in range(len(df)):
        item_no = df[i]['item_no']
        title = df[i]['title']
        pattern = rf"{re.escape(item_no.lower())}[\s.]*{re.escape(title.lower())}"
        j=0
        done = False
        while (not done) and (j<len(agenda_text_lines)):
            line = agenda_text_lines[j].lower()
            if re.search(pattern, line):
                df[i]['start_line'] = j
                done = True
            elif re.search(pattern, re.sub(r'\s+', ' ', line).strip()):
                df[i]['start_line'] = j
                done = True
            else:
                # check if all non-stopwords in title are contained in line
                line_nopunc = re.sub(r'[^\w\s]', '', line)
                title_nopunc = re.sub(r'[^\w\s]', '', title)
                line_tokens = [tok for tok in line_nopunc.lower().split() if tok not in stop_words]
                title_tokens = [tok for tok in title_nopunc.lower().split() if tok not in stop_words]
                no_in_line = (len(line_tokens)>0) and (f"{item_no}"==line_tokens[0])
                title_in_line = set(title_tokens) <= set(line_tokens)
                if (no_in_line and title_in_line):
                    df[i]['start_line'] = j
                    done = True
            j+=1
        if j==len(agenda_text_lines):
            print("error")
            print(item_no, title)
            raise RuntimeError("Line containing agenda item not found")
    # Check if start line is monotonic
    start_line = 0
    for i in range(len(df)):
        if df[i]['start_line'] < start_line:
            raise RuntimeError("start lines not monotonic")
        start_line = df[i]['start_line']
    # Find the end line of each item
    for i in range(len(df)):
        if i==len(df)-1:
            j=df[i]['start_line']+1
            done = False
            while (not done) and (j<len(agenda_text_lines)):
                line = agenda_text_lines[j]
                if ('next' in line.lower()) and ('meeting' in line.lower()) and ('city' in line.lower()) and ('planning' in line.lower()) and ('commission' in line.lower()):
                    df[i]['end_line'] = j-1
                    done = True
                j+=1
            if j==len(agenda_text_lines):
                raise RuntimeError("Line containing next city planning commission meeting not found")
        else:
            df[i]['end_line'] = df[i+1]['start_line']-1
    # Extract the content of each item
    for i in range(len(df)):
        start_line = df[i]['start_line']
        end_line = df[i]['end_line']
        content = '\n'.join(agenda_text_lines[start_line:end_line+1])
        df[i]['content'] = content
    
    df = pd.DataFrame.from_dict(df)
    df.to_pickle(f"../../intermediate_data/cpc/{year}/{date}/agenda-items.pkl")


2018-05-10... 2018-05-23... 2018-06-14... 2018-07-12... 2018-07-26... 2018-08-09... 2018-08-23... 2018-09-13... 2018-09-27... 2018-10-11... 2018-10-25... 2018-11-08... 2018-11-29... 2018-12-13... 2018-12-20... 2019-01-10... 2019-01-24... 2019-02-14... 2019-02-28... 2019-03-14... 2019-03-28... 2019-04-11... 2019-05-09... 2019-05-23... 2019-06-13... 2019-06-27... 2019-07-11... 2019-07-25... 2019-08-08... 2019-08-22... 2019-09-12... 2019-09-26... 2019-10-10... 2019-10-24... 2019-11-14... 2019-11-21... 2019-12-12... 2019-12-19... 2020-01-09... 2020-01-23... 2020-02-13... 2020-03-12... 2020-04-23... 2020-05-14... 2020-05-28... 2020-06-04... 2020-06-11... 2020-06-25... 2020-07-09... 2020-07-23... 2020-08-13... 2020-08-27... 2020-09-10... 2020-09-17... 2020-09-24... 2020-10-08... 2020-10-15... 2020-10-22... 2020-11-05... 2020-11-19... 2020-12-03... 2020-12-10... 2020-12-17... 2021-01-14... 2021-01-28... 2021-02-11... 2021-02-18... 2021-02-25... 2021-03-11... 2021-03-25... 2021-04-08... 2021-0

In [15]:
date = meetings_meta_df.sample(1).iloc[0]['date']
year = date[0:4]
df = pd.read_pickle(f"../../intermediate_data/cpc/{year}/{date}/agenda-items.pkl")
df

,year,date,item_no,title,summary,start_line,end_line,content
0,2022,2022-08-11,1,Determination to Continue Holding Meetings via...,The commission will decide whether to continue...,81,86,1. DETERMINATION TO CONTINUE HOLDING MEE...
1,2022,2022-08-11,2,Director’s Report and Commission Business,The commission will receive updates on legal a...,87,98,2. DIRECTOR’S REPORT AND COMMISSION BUSI...
2,2022,2022-08-11,3,Neighborhood Council Position Statements on Ag...,Neighborhood Council representatives will pres...,99,106,3. NEIGHBORHOOD COUNCIL POSITION STATEME...
3,2022,2022-08-11,4,General Public Comment,The commission will provide a 30-minute opport...,107,123,4. GENERAL PUBLIC COMMENT ...
4,2022,2022-08-11,5,Reconsiderations,The commission may reconsider actions on previ...,124,133,5. RECONSIDERATIONS ...
5,2022,2022-08-11,6,Consent Calendar (No Items),"There are no items on the consent calendar, wh...",134,140,6. CONSENT CALENDAR (No Items) ...
6,2022,2022-08-11,7,CPC-2020-1946-GPA-ZC-HD-CU-SPR-RDP,The applicant requests a continuation to Augus...,141,195,7. CPC-2020-1946-GPA-ZC-HD-CU-SPR-RDP ...
7,2022,2022-08-11,8,DIR-2017-4167-PUB-CDP-SPP-1A,An appeal of a Planning Director’s Determinati...,196,250,8. DIR-2017-4167-PUB-CDP-SPP-1A ...
8,2022,2022-08-11,9,ZA-2017-3950-ZAA-1A,An appeal of a Planning Director’s Determinati...,251,291,9. ZA-2017-3950-ZAA-1A ...
9,2022,2022-08-11,10,DIR-2017-4173-CDP-SPP-1A,An appeal of a Planning Director’s Determinati...,292,343,10. DIR-2017-4173-CDP-SPP-1A ...
